<img src = "https://raw.githubusercontent.com/PauliBCN/Capstone/master/Toronto.JPG" width = 350>
<h1 align=center><font size = 5>Toronto</font></h1>

In [6]:
import requests
import lxml.html as lh
import pandas as pd

In [7]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)
df[0].head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [8]:
df2 = df[0]
df2 = df2[df2.Borough != 'Not assigned']
for i, row in df2.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']
df2.head(10)

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [9]:
#df3=df2['Postcode','Borough','Neighborhood']
df3=df2.groupby("Postcode")["Neighborhood"].agg(",".join).reset_index(name='new')
df3 = df3.rename(columns={'new':'Neighborhood'})
df3.head(10)

,Postcode,Neighborhood
0,M1B,"Rouge,Malvern"
1,M1C,"Highland Creek,Rouge Hill,Port Union"
2,M1E,"Guildwood,Morningside,West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae
5,M1J,Scarborough Village
6,M1K,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,"Clairlea,Golden Mile,Oakridge"
8,M1M,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,"Birch Cliff,Cliffside West"


In [39]:
df3.count()

Postcode        103
Neighborhood    103
dtype: int64

In [33]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_506e9a640e4f43c686655e00d1f11327 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='P8PCW9lkYr7PfhtnHQhPKiueDqd2b_kXKJT70l_gIbPr',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_506e9a640e4f43c686655e00d1f11327.get_object(Bucket='toronto-donotdelete-pr-df849j0vsvtl6b',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_coord = pd.read_csv(body)
#df_coord.head(20)

In [57]:
df_col = pd.concat([df_coord,df3], axis=1)
df_col=df_col.drop(['Postal Code'], axis=1)
df_col.head()
#es la mejor evrsion

,Latitude,Longitude,Postcode,Neighborhood
0,43.806686,-79.194353,M1B,"Rouge,Malvern"
1,43.784535,-79.160497,M1C,"Highland Creek,Rouge Hill,Port Union"
2,43.763573,-79.188711,M1E,"Guildwood,Morningside,West Hill"
3,43.770992,-79.216917,M1G,Woburn
4,43.773136,-79.239476,M1H,Cedarbrae


In [55]:
df_col.count()

Postal Code     103
Latitude        103
Longitude       103
Neighborhood    103
dtype: int64

In [50]:
df4=df2
df4=df4.drop(['Neighborhood'], axis=1)
df4 = df4[df4.Borough != 'NaN']
df4.head()
#ok

,Postcode,Borough
2,M3A,North York
3,M4A,North York
4,M5A,Downtown Toronto
5,M6A,North York
6,M6A,North York


In [62]:
df_final=df_col
de_final=df_final.merge(df4, how='left')
df_final.head()
#no agrega borou

,Latitude,Longitude,Postcode,Neighborhood
0,43.806686,-79.194353,M1B,"Rouge,Malvern"
1,43.784535,-79.160497,M1C,"Highland Creek,Rouge Hill,Port Union"
2,43.763573,-79.188711,M1E,"Guildwood,Morningside,West Hill"
3,43.770992,-79.216917,M1G,Woburn
4,43.773136,-79.239476,M1H,Cedarbrae


In [71]:
dfproj=pd.DataFrame(df_final)
dfproj.head()

,Latitude,Longitude,Postcode,Neighborhood
0,43.806686,-79.194353,M1B,"Rouge,Malvern"
1,43.784535,-79.160497,M1C,"Highland Creek,Rouge Hill,Port Union"
2,43.763573,-79.188711,M1E,"Guildwood,Morningside,West Hill"
3,43.770992,-79.216917,M1G,Woburn
4,43.773136,-79.239476,M1H,Cedarbrae


In [74]:
df9=dfproj
df9=df9.merge(df4, how='left')
df9.head()

,Latitude,Longitude,Postcode,Neighborhood,Borough
0,43.806686,-79.194353,M1B,"Rouge,Malvern",Scarborough
1,43.806686,-79.194353,M1B,"Rouge,Malvern",Scarborough
2,43.784535,-79.160497,M1C,"Highland Creek,Rouge Hill,Port Union",Scarborough
3,43.784535,-79.160497,M1C,"Highland Creek,Rouge Hill,Port Union",Scarborough
4,43.784535,-79.160497,M1C,"Highland Creek,Rouge Hill,Port Union",Scarborough


In [83]:
df10 = df9.drop_duplicates(keep="last")
df10.head()
#count()

,Latitude,Longitude,Postcode,Neighborhood,Borough
1,43.806686,-79.194353,M1B,"Rouge,Malvern",Scarborough
4,43.784535,-79.160497,M1C,"Highland Creek,Rouge Hill,Port Union",Scarborough
7,43.763573,-79.188711,M1E,"Guildwood,Morningside,West Hill",Scarborough
8,43.770992,-79.216917,M1G,Woburn,Scarborough
9,43.773136,-79.239476,M1H,Cedarbrae,Scarborough
